<a href="https://colab.research.google.com/github/gretiere545/langdeck/blob/main/langdeck_sync.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Langdeck Synchro de bout en bout

In [1]:
#!/usr/bin/env python
# -*- coding: utf8 -*-
!pip install gspread --upgrade
# fix here => https://stackoverflow.com/questions/71347973/modulenotfounderror-no-module-named-gspread-models
!pip uninstall gspread-dataframe
!pip install gspread-dataframe
!pip install gspread-formatting
!pip install airtable-python-wrapper
import pandas as pd
import numpy as np
import requests
import json
import uuid
from datetime import datetime
import datetime
pd.set_option("display.width",1000)
# Ce bout de code pour pouvoir downloader des fichiers sur le drive
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/Trad-Union/Corpus/ASAMLA
from google.colab import files

# Ce bout de code pour pouvoir downloader des google sheets dans des dataframes
from google.colab import auth
auth.authenticate_user()
import gspread
from gspread_dataframe import get_as_dataframe, set_with_dataframe
from gspread_formatting import *
from oauth2client.client import GoogleCredentials
# fix here => https://github.com/burnash/gspread/issues/1014
from google.auth import default
creds, _ = default()
gc = gspread.authorize(creds)
#gc = gspread.authorize(GoogleCredentials.get_application_default())
from airtable import Airtable

!pip install txtai[pipeline]
!pip install sacremoses
from txtai.pipeline import Translation
translate = Translation()

!pip install -U deep-translator

!pip install pymediawiki
# https://github.com/barrust/mediawiki
# https://pymediawiki.readthedocs.io/en/latest/code.html
from mediawiki import MediaWiki
!pip install --upgrade deepl
!export PYTHONIOENCODING=utf8


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Attempting uninstall: gspread
    Found existing installation: gspread 3.4.2
    Uninstalling gspread-3.4.2:
      Successfully uninstalled gspread-3.4.2
Found existing installation: gspread-dataframe 3.0.8
Uninstalling gspread-dataframe-3.0.8:
  Would remove:
    /usr/local/lib/python3.7/dist-packages/gspread_dataframe-3.0.8.dist-info/*
    /usr/local/lib/python3.7/dist-packages/gspread_dataframe.py
Proceed (y/n)? y
  Successfully uninstalled gspread-dataframe-3.0.8
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Mounted at /content/drive
/content/drive/MyDrive/Trad-Union/Corpus/ASAMLA
Looking in indexes: https://pypi.org/simple, https://us-

# Configuration

In [2]:
# Code ISO
iso_codes = {"ams": "ar",
            "rus": "ru",
            "ukr": "uk",
            "rou": "ro",
            "alb": "sq",
            "eng": "en",
            "tur": "tr",
            "tig": "ti",
            "geo": "ka",
            "arm": "hy",
            "dar": "fa",
            "prs": "fa",
            "pst": "ps",
            "esp": "es",
            "pol": "pl",
            "som": "so",
            "amh": "am",
            "all": "de",
            "hun": "hu" }
# IA
dict_vendor =[
    {"vendor-code":"11", "vendor-abbv":"neuml","vendor-name":"txtai(Neuml)","pivot":"en", "f":"fetch_deep_tlr_txtai"},
    {"vendor-code":"12", "vendor-abbv":"google","vendor-name":"GoogleTranslator","pivot":"en", "f":"fetch_deep_tlr_google"},
    {"vendor-code":"13", "vendor-abbv":"pons","vendor-name":"Pons","pivot":"fr", "f":"fetch_deep_tlr_pons"},   
    {"vendor-code":"14", "vendor-abbv":"deepl","vendor-name":"DeepL","pivot":"en", "f":"fetch_deep_tlr_deepl"}
]            


In [3]:
dict_lang_vendor = [{"language":"ams", "iso-code":"ar", "vendor-code":["12"]},
            {"language":"rus", "iso-code":"ru", "vendor-code":["12"]},
            {"language":"ukr", "iso-code":"uk", "vendor-code":["12"]},
            {"language":"rou", "iso-code":"ro", "vendor-code":["12","14"]},
            {"language":"alb", "iso-code":"sq", "vendor-code":["12"]},
            {"language":"eng", "iso-code":"en", "vendor-code":["12"]},
            {"language":"tur", "iso-code":"tr", "vendor-code":["12"]},
            {"language":"tig", "iso-code":"ti", "vendor-code":["12"]},
            {"language":"geo", "iso-code":"ka", "vendor-code":["12"]},
            {"language":"arm", "iso-code":"hy", "vendor-code":["12"]},
            {"language":"dar", "iso-code":"fa", "vendor-code":["12"]},
            {"language":"prs", "iso-code":"fa", "vendor-code":["12"]},
            {"language":"pst", "iso-code":"ps", "vendor-code":["12"]},
            {"language":"esp", "iso-code":"es", "vendor-code":["12"]},
            {"language":"pol", "iso-code":"pl", "vendor-code":["12"]},
            {"language":"som", "iso-code":"so", "vendor-code":["12"]},
            {"language":"amh", "iso-code":"am", "vendor-code":["12"]},
            {"language":"all", "iso-code":"de", "vendor-code":["12"]},
            {"language":"hun", "iso-code":"hu", "vendor-code":["12","14"]}]

In [4]:
from IPython.core.debugger import exception_colors
def get_cc_config(f):
  vk = []
  try:
    with open(f) as vk_f:
        vk = json.load(vk_f)
  except BaseException as e:
    print(e)
  return vk

def set_cc_config (vk, f):
  # sauvegarde du fichier de paramétrage
  with open(f, 'w') as fp:
      json.dump(vk, fp)  

#
# Ouverture de la Sheet Centrale (corpus_central_base)
#
def get_CCDB_wb(uri):
  sheet_central = uri
  wb_central = gc.open_by_url(sheet_central)
  return wb_central

def get_CCDB_data(wb, idx):
  t_corpus = wb.get_worksheet(idx)
  data_t_corpus = t_corpus.get_all_values()
  return data_t_corpus

#
# DataFrame CC global (toutes les langues)
#
def get_ccdf_global(CCDB):
  return pd.DataFrame(CCDB[1:], columns=CCDB[0])

#
# DataFrame CC Français (invariable)
#
def get_ccdf_fr(df_cc):
  # on supprime les doublons
  return df_cc[['uid','expression','theme','état','date','commentaires','index']]    

# Comparaison des versions de référentiels
def getLastModifiedTime (wb):
  # https://stackoverflow.com/questions/64254039/python-how-to-check-if-a-google-spreadsheets-was-updated
  # https://stackoverflow.com/questions/52260789/update-googlesheet-cell-with-timestamp-from-python
  revisions_uri = f'https://www.googleapis.com/drive/v3/files/{wb.id}/revisions'
  headers = {'Authorization': f'Bearer {GoogleCredentials.get_application_default().get_access_token().access_token}'}
  response = requests.get(revisions_uri, headers=headers).json()
  return response['revisions'][-1]['modifiedTime']    

def txtai_alpha (input, iso_code):
  try:
    translation = translate(input, iso_code)
  except Exception as e:
    translation = ""
    pass
  return translation, 2

def airtable_fetch_all_records():
  vk_at = airtable.get_all(view='Admin',sort='vocabulary_unit')
  df_vkat = pd.DataFrame.from_records((r['fields'] for r in vk_at))
  return df_vkat

def fetch_pivot_en (txt_fr):
  try:
    translation = translate(txt_fr, "en")
    print (translation)
  except Exception as e:
    translation = ""
    pass  
  return translation

def make_unique_id():
  return str(uuid.uuid4())[:8]  

def chk_uid (vk_uid, record):
  try:
    value = record['fields']['uid']
  except KeyError:
    # uid manquant
    print ("mssing uid")
    while True:
      auid = make_unique_id()
      if auid not in vk_uid:
        aDict = {}
        aDict["uid"] = auid
        print (record['id'] + " " + aDict["uid"])
        airtable.update(record['id'], aDict)
        vk_uid.append(auid) 
        break 
  finally:
    return

def airtable_check_missing_uids():
  vk_uid = df_vkat['uid'].tolist()
  for page in airtable.get_iter(view='Admin',sort='vocabulary_unit'):
    for record in page:
      chk_uid (vk_uid, record)

def compare_langdeck_time(hash, time):
  ts1 = time
  ts2 = df_langdeck.loc[df_langdeck["hash"]==hash]["unixtime"].values[0]
  return ts1 > ts2

def compare_airtable_time(hash, time):
  ts1 = time
  ts2 = df_airtable_for_langdeck.loc[df_airtable_for_langdeck["hash"]==hash]["unixtime"].values[0]
  return ts1 > ts2

def get_df_airtable_for_langdeck():
  df_vkat = airtable_fetch_all_records()
  df_temp = df_vkat.copy()
  df_temp = df_temp[["uid","lemma","rank","vocabulary_unit","pivot","alphabet","theme","timestamp","ams","eng","tur","rus","ukr","rou","hun","alb","geo","arm","dar","pst","prs","esp","pol","tig"]]
  #df_temp.rename(columns={"UID":"uid","Proposition":"expression","pivot_en":"pivot","idx_cor":"idx", "Maitre":"maitre","Ordre":"ordre","Thématique":"themes","lastModified":"timestamp"}, inplace=True)
  # https://stackoverflow.com/questions/28654047/convert-columns-into-rows-with-pandas
  df_airtable_for_langdeck = pd.melt(df_temp, id_vars=["uid", "lemma","rank","vocabulary_unit","pivot","alphabet","timestamp","theme"], 
                    var_name="language", value_name="translation")
  #https://stackoverflow.com/questions/60393668/pandas-generate-unique-id-based-on-row-values
  #https://stackoverflow.com/questions/64604408/transform-dataframe-column-with-a-hash-value
  df_airtable_for_langdeck['hash'] = df_airtable_for_langdeck[['uid', 'language']].sum(axis=1).astype(str)
  #Fonction lambda pour récupérer le nom long de la langue (https://stackoverflow.com/questions/2191699/find-an-element-in-a-list-of-tuples)
  df_airtable_for_langdeck["language_fulltext"] = df_airtable_for_langdeck["language"].apply(lambda x:[item for item in [(a_dict["trigramme"],a_dict["language"]) for a_dict in vk_languages] if item[0] == x][0][1])  
  df_airtable_for_langdeck['timestamp'] = df_airtable_for_langdeck['timestamp'].astype('datetime64[s]')
  df_airtable_for_langdeck['unixtime'] = df_airtable_for_langdeck['timestamp'].astype('datetime64[s]').astype('int')
  # ajout des thématiques  
  # on éclate la liste des thèmes en 3 colonnes
  df_airtable_for_langdeck[["theme_1","theme_2","theme_3"]] = pd.DataFrame([pd.Series(x) for x in df_airtable_for_langdeck.theme])
  return df_airtable_for_langdeck    

def get_df_langdeck(url):
  # Ouverture de la Sheet Langdeck (mots à traduire)
  sheet = url
  wb  = gc.open_by_url(sheet)
  t_corpus = wb.worksheet("t_corpus_all_entries")
  data_t_corpus = t_corpus.get_all_values()
  df_langdeck = pd.DataFrame(data_t_corpus[1:], columns=data_t_corpus[0]).drop_duplicates()  
  df_langdeck['ordre'] = pd.to_numeric(df_langdeck['ordre'])
  df_langdeck['timestamp'] = df_langdeck['timestamp'].astype('datetime64[s]')
  df_langdeck['unixtime'] = df_langdeck['timestamp'].astype('datetime64[s]').astype('int')
  return df_langdeck

def is_audio(f):
  path = "/content/drive/My Drive/appsheet/data/langdeck-1629444/"
  f = path + f
  result = True
  try:
    file = open(f, "r")
  except FileNotFoundError as e:
    # do nothing
    result = False
  return result  

def get_translation_pivot (hash):
  r = ""
  try:
    r = df_airtable_for_langdeck.loc[df_airtable_for_langdeck["hash"]==hash]["pivot"].values.item()
  except Exception as e:
    print (hash)
    pass
  return r  

def get_wikipedia(text, language):
  wikipedia = MediaWiki(lang=language)  
  r = ""
  try:
    p = wikipedia.page(text)
    r = p.summary
  except Exception as e:
    r = str(e)
    pass
  return r  

In [5]:
# Config Langues
config_languages = 'med_vac_synthese.json'
vk_languages = get_cc_config(config_languages)

params = ()
api_key = "keyrI98TIqu6mbFcf" 
api_key_gmail = "keysgpJWq3aWwqSZe"
headers = {"Authorization": "Bearer " + api_key,"Content-Type" : "application/json"}
base_id = "appawT3gg7cJhPYv0" # Base Nouvelle HV
table_name = 'Corpus'

db_langdeck_url = "https://docs.google.com/spreadsheets/d/1iJLt3qAKq576eNirMwcsdRltg7nerwa9N70dA0vraa8/" #db-langdeck

# Chargement table Airtable
airtable = Airtable(base_id, table_name, api_key)
# Vers Dataframe
df_vkat = airtable_fetch_all_records()

# maj des uid manquants
airtable_check_missing_uids()

# Vérification qu'il n'existe pas des doublons (hash)
df_airtable_for_langdeck = get_df_airtable_for_langdeck()
df_airtable_for_langdeck[df_airtable_for_langdeck['hash'].duplicated()].sort_values("hash")
# on éclate la liste des thèmes en 3 colonnes
df_airtable_for_langdeck[["theme_1","theme_2","theme_3"]] = pd.DataFrame([pd.Series(x) for x in df_airtable_for_langdeck.theme])
# on ajoute une colonne de translation state (0=VOID; 1=PENDING; 2=OK; 3=KO)
df_airtable_for_langdeck["translation_state"] = df_airtable_for_langdeck["translation"].apply(lambda x:0 if pd.isna(x) else 1)

# ouverture base Langdeck (t_corpus_all_entries)
df_langdeck = get_df_langdeck(db_langdeck_url)
df_langdeck['is_audio'] = df_langdeck['audio'].apply(lambda x:is_audio(x))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:124: FutureWarning: casting datetime64[ns] values to int64 with .astype(...) is deprecated and will raise in a future version. Use .view(...) instead.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:139: FutureWarning: casting datetime64[ns] values to int64 with .astype(...) is deprecated and will raise in a future version. Use .view(...) instead.


## Comparaison 1 : new records dans Airtable à ajouter dans Langdeck

In [6]:
sheet = db_langdeck_url
wb  = gc.open_by_url(sheet)
# table Corpus
t_corpus = wb.worksheet("t_corpus_all_entries")
# table Lexique : group par thème
# table Lemma

In [7]:
# la différence se fait sur le hash et non l'uid
df_new_in_airtable = df_airtable_for_langdeck[~df_airtable_for_langdeck["hash"].isin(df_langdeck["hash"])]
print (str(len(df_new_in_airtable)) + " new airtable rows detected.")
for index, row in df_new_in_airtable.iterrows():

  if pd.isna(row['theme_1']):
    row['theme_1'] = ""
  if pd.isna(row['theme_2']):
    row['theme_2'] = ""
  if pd.isna(row['theme_3']):
    row['theme_3'] = ""        
  print (row['hash'])    
  t_corpus.append_row([row['hash'],
                       row['uid'], 
                       row['lemma'], 
                       row['rank'],
                       row['vocabulary_unit'], 
                       row['alphabet'], 
                       str(pd.to_datetime(row['timestamp'])), 
                       row['language'],                     
                       "", 
                       row['language_fulltext'],
                       "Files_Files_/Corpus/t_expo_audio/" + row['uid'] + "-" + row['language'] + ".mp3",
                       False,
                       "",
                       row['theme_1'],
                       row['theme_2'],
                       row['theme_3']                     
                       ])

# on recharge avec les nouvelles données
df_langdeck = get_df_langdeck(db_langdeck_url)

0 new airtable rows detected.


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:139: FutureWarning: casting datetime64[ns] values to int64 with .astype(...) is deprecated and will raise in a future version. Use .view(...) instead.


Mise à jour (INIT) des thématiques

In [8]:
df_airtable_for_langdeck = get_df_airtable_for_langdeck()
df_airtable_for_langdeck[df_airtable_for_langdeck['hash'].duplicated()].sort_values("hash")

# On met à jour les méta mais pas les traductions (traduction dans langdeck uniq.)
for index, row in df_airtable_for_langdeck.iterrows():
  # clé A et B : concat des clés primaires
  t = df_langdeck.loc[df_langdeck["hash"]==row['hash']].applymap(str)
  # key_b = langdeck (sheet)
  try:
    key_b = t["theme_1"].item() + t["theme_2"].item() + t["theme_3"].item()
  except Exception as e:
    print (row['hash'] + str(e))
    pass

  if not pd.isna(row['theme_1']):
    key_a = row['theme_1']
  else:
    row['theme_1'] = ""
    key_a = row['theme_1']

  if not pd.isna(row['theme_2']): 
    key_a = key_a + row['theme_2']  
  else:
    row['theme_2'] = ""
    key_a = key_a + row['theme_2']        
      
  if not pd.isna(row['theme_3']):
    key_a = key_a + row['theme_3']  
  else:
    row['theme_3'] = ""    
    key_a = key_a + row['theme_3'] 

  # si lex clés diffenet
  
  if key_a != key_b:  
    cell = t_corpus.find(row['hash'])
    print ("Updating cell " + str(cell.row))
    t_corpus.update_cell(cell.row, 14, row['theme_1'])
    t_corpus.update_cell(cell.row, 15, row['theme_2'])
    t_corpus.update_cell(cell.row, 16, row['theme_3'])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:124: FutureWarning: casting datetime64[ns] values to int64 with .astype(...) is deprecated and will raise in a future version. Use .view(...) instead.


In [9]:
df_langdeck = get_df_langdeck(db_langdeck_url)
df_airtable_for_langdeck["last_updated"] = df_airtable_for_langdeck[["hash","unixtime"]].apply(lambda x:compare_langdeck_time(x[0],x[1]), axis=1) 
df_newly_updated_in_airtable = df_airtable_for_langdeck.loc[df_airtable_for_langdeck["last_updated"]]

sheet = db_langdeck_url
wb  = gc.open_by_url(sheet)
t_corpus = wb.worksheet("t_corpus_all_entries")
# on recharge avec les nouvelles données
df_langdeck = get_df_langdeck(db_langdeck_url)

for index, row in df_newly_updated_in_airtable.iterrows():
  # clé A et B : concat des clés primaires
  key_a = row['lemma']+str(row["rank"])+row["vocabulary_unit"]+row["alphabet"]
  t = df_langdeck.loc[df_langdeck["hash"]==row['hash']].applymap(str)
  key_b = t["maitre"].item() + t["ordre"].item() + t["expression"].item() + t["idx"].item()
  # si lex clés diffenet
  if key_a != key_b:
    cell = t_corpus.find(row['hash'])
    t_corpus.update_cell(cell.row, 3, row['lemma'])
    t_corpus.update_cell(cell.row, 4, row['rank'])  
    t_corpus.update_cell(cell.row, 5, row['vocabulary_unit'])  
    t_corpus.update_cell(cell.row, 6, row['alphabet'])
    t_corpus.update_cell(cell.row, 7, str(pd.to_datetime(row['timestamp'])))    
    print ("Update from Airtable Meta " + key_a)
# on recharge avec les nouvelles données
df_langdeck = get_df_langdeck(db_langdeck_url)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:139: FutureWarning: casting datetime64[ns] values to int64 with .astype(...) is deprecated and will raise in a future version. Use .view(...) instead.


## Comparaison 3 : updated records from Langdeck to Airtable

In [10]:
# liste des enregistrements modifiés dans Langdeck (synchro Langdeck -> Airtable)
df_langdeck["last_updated"] = df_langdeck[["hash","unixtime"]].apply(lambda x:compare_airtable_time(x[0],x[1]), axis=1) 
df_newly_updated_in_langdeck = df_langdeck.loc[df_langdeck["last_updated"]]
df_newly_updated_in_langdeck

,hash,uid,maitre,ordre,expression,idx,timestamp,language,translation,language_fulltext,...,theme_1_svg,theme_2_svg,theme_3_svg,unixtime,translation_state,translation_pivot,translation_ai_source,translation_score,translation_state_txt,last_updated
342,6e65dde7eng,6e65dde7,Abcès,1,Abcès,A,2022-08-25 09:17:53,eng,Abscess,Anglais,...,,,,1661419073000000000,1,Abscesses,0,1,OK,True
1524,b6b565a5ukr,b6b565a5,Immunité,1,Immunité,I,2022-05-20 20:43:25,ukr,ІМунітет,Ukrainien,...,,,,1653079405000000000,2,Immunity,0,0.5,Pending,True
2388,6e65dde7alb,6e65dde7,Abcès,1,Abcès,A,2022-08-25 09:20:21,alb,abscesi,Albanais,...,,,,1661419221000000000,2,Abscesses,0,0.5,Pending,True
2496,6880482balb,6880482b,Doigt,1,Doigt,D,2022-08-24 11:27:07,alb,,Albanais,...,,,,1661340427000000000,3,Finger,0,0,Reject,True
3070,6e65dde7arm,6e65dde7,Abcès,1,Abcès,A,2022-08-25 09:15:16,arm,,Arménien,...,,,,1661418916000000000,3,Abscesses,0,0,Reject,True
3411,6e65dde7dar,6e65dde7,Abcès,1,Abcès,A,2022-08-25 09:16:34,dar,,Dari,...,,,,1661418994000000000,3,Abscesses,0,0,Reject,True


In [11]:
## Mise à jour dans Airtable 
for index, row in df_newly_updated_in_langdeck.iterrows():
  if row["uid"] is not None:
    record = airtable.match('uid', row["uid"])
    if bool(record) is True:
      aDict = {}
      aDict[row["language"]] = row["translation"]
      airtable.update(record['id'], aDict)
      print ("Updating status for " + row["uid"])

# + maj symétrique du timestamp dans Langdeck
for index, row in df_newly_updated_in_langdeck.iterrows():
  if row["uid"] is not None:
    record = airtable.match('uid', row["uid"])
    if bool(record) is True:
      # mise à jour symétrique pour garantir le même timestamp
      cell = t_corpus.find(row['hash'])
      t_corpus.update_cell(cell.row, 6, row["idx"])
      t_corpus.update_cell(cell.row, 7, str(pd.to_datetime(row['timestamp'])))

# on recharge avec les nouvelles données
df_langdeck = get_df_langdeck(db_langdeck_url)
# on ajoute une colonne pour la translation pivot (en)
df_langdeck["translation_pivot"] = df_langdeck["hash"].apply(lambda x:get_translation_pivot(x))

Updating status for 6e65dde7
Updating status for b6b565a5
Updating status for 6e65dde7
Updating status for 6880482b
Updating status for 6e65dde7
Updating status for 6e65dde7


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:139: FutureWarning: casting datetime64[ns] values to int64 with .astype(...) is deprecated and will raise in a future version. Use .view(...) instead.


# Table Corpus Headword (Lemme)
Indexé sur le Lemme

In [12]:
# table vue Lemme
at_lemme = Airtable(base_id, "Corpus", api_key)

In [13]:
vk_lemme = at_lemme.get_all(view='Lemme',sort='vocabulary_unit')
df_vklemme = pd.DataFrame.from_records((r['fields'] for r in vk_lemme))

In [14]:
### on charge la table t_corpus_head
sheet = db_langdeck_url
wb  = gc.open_by_url(sheet)
tbl = wb.worksheet("t_corpus_head")
# on recharge avec les nouvelles données
data = tbl.get_all_values()
df_corpus_terme_prefer = pd.DataFrame(data[1:], columns=data[0]).drop_duplicates(subset=['terme_pref'])  

In [15]:
df_diff = df_langdeck[~df_langdeck["maitre"].isin(df_corpus_terme_prefer["terme_pref"])].drop_duplicates(subset=['maitre'])

Différences entre Langdeck REf et termes_pref

In [16]:
# Différences entre Corpus et Lemme
df_diff = df_langdeck[~df_langdeck["maitre"].isin(df_corpus_terme_prefer["terme_pref"])].drop_duplicates(subset=['maitre'])
df_diff = df_diff[["maitre","idx","translation_pivot"]]
df_diff.rename(columns={"maitre":"terme_pref"}, inplace=True)

frames =[]
frames.append(df_corpus_terme_prefer)
frames.append(df_diff)
df_vocabulary = pd.concat(frames)

In [17]:
df_vocabulary

,terme_pref,idx,translation_pivot,wikipedia_fr,wikipedia_en,wiki_search_text_fr,wiki_search_text_en,wikipedia_fr_state,wikipedia_en_state
0,AME,A,AME- State Medical Aid,"L’âme (du latin anima, « souffle, respiration ...",The French Universal Health Coverage (French: ...,AME,AME- State Medical Aid,0,0
1,Abcès,A,Abscesses,Un abcès est une accumulation locale de pus ap...,An abscess is a collection of pus that has bui...,Abcès,Abscesses,0,0
2,Abdomen,A,Abdomen,"L’abdomen (du latin, de même sens) est une par...","The abdomen (colloquially called the belly, tu...",Abdomen,Abdomen,0,0
3,Accident,A,Accident,"Un accident est un événement, généralement non...","An accident is an unintended, normally unwante...",Accident,Accident,0,0
4,Accouchement,A,Childbirth,L'accouchement est l'action de mettre un enfan...,"Childbirth, also known as labour or delivery, ...",Accouchement,Childbirth,0,0
...,...,...,...,...,...,...,...,...,...
233,Acouphènes,A,Tinnitus,"Un acouphène (du grec ancien ἀκούω / akoúô, «...",,Acouphènes,Tinnitus,0,0
234,Hormone,H,Hormone,Une hormone est une substance produite de faço...,,Hormone,Hormone,0,0
235,Intolérance,I,Tolerance,"\n""Intolérance"" may refer to: \n Intolerance\...",,Intolérance,Tolerance,1,0
236,Prothèse,P,Prosthesis,"\n""Prothèse"" may refer to: \n Prothèse\n Rit...",,Prothèse,Prosthesis,1,0


In [18]:
# traduction en anglais (pivot)
from google.colab import output
output.enable_custom_widget_manager()
df_vocabulary["translation_pivot"] = df_vocabulary[["terme_pref","translation_pivot"]].apply(lambda x:fetch_pivot_en(x[0]) if x[1]=="" else x[1], axis=1)


In [19]:

df_vocabulary["wikipedia_fr"] = df_vocabulary[["terme_pref","wikipedia_fr"]].apply(lambda x:get_wikipedia(x[0], "fr") if pd.isna(x[1]) else x[1], axis=1)
df_vocabulary["wikipedia_en"] = df_vocabulary[["translation_pivot","wikipedia_en"]].apply(lambda x:get_wikipedia(x[0], "en") if pd.isna(x[1]) else x[1], axis=1)

In [20]:
df_vocabulary["wiki_search_text_fr"] = df_vocabulary["terme_pref"]
df_vocabulary["wiki_search_text_en"] = df_vocabulary["translation_pivot"]

In [21]:
df_vocabulary

,terme_pref,idx,translation_pivot,wikipedia_fr,wikipedia_en,wiki_search_text_fr,wiki_search_text_en,wikipedia_fr_state,wikipedia_en_state
0,AME,A,AME- State Medical Aid,"L’âme (du latin anima, « souffle, respiration ...",The French Universal Health Coverage (French: ...,AME,AME- State Medical Aid,0,0
1,Abcès,A,Abscesses,Un abcès est une accumulation locale de pus ap...,An abscess is a collection of pus that has bui...,Abcès,Abscesses,0,0
2,Abdomen,A,Abdomen,"L’abdomen (du latin, de même sens) est une par...","The abdomen (colloquially called the belly, tu...",Abdomen,Abdomen,0,0
3,Accident,A,Accident,"Un accident est un événement, généralement non...","An accident is an unintended, normally unwante...",Accident,Accident,0,0
4,Accouchement,A,Childbirth,L'accouchement est l'action de mettre un enfan...,"Childbirth, also known as labour or delivery, ...",Accouchement,Childbirth,0,0
...,...,...,...,...,...,...,...,...,...
233,Acouphènes,A,Tinnitus,"Un acouphène (du grec ancien ἀκούω / akoúô, «...",,Acouphènes,Tinnitus,0,0
234,Hormone,H,Hormone,Une hormone est une substance produite de faço...,,Hormone,Hormone,0,0
235,Intolérance,I,Tolerance,"\n""Intolérance"" may refer to: \n Intolerance\...",,Intolérance,Tolerance,1,0
236,Prothèse,P,Prosthesis,"\n""Prothèse"" may refer to: \n Prothèse\n Rit...",,Prothèse,Prosthesis,1,0


In [22]:
df_vocabulary["wikipedia_fr_state"]=df_vocabulary["wikipedia_fr"].apply(lambda x: 1 if (len(x)>0 and x[0]=="\n") else 0)
df_vocabulary["wikipedia_en_state"]=df_vocabulary["wikipedia_en"].apply(lambda x: 1 if (len(x)>0 and x[0]=="\n") else 0)

In [23]:
try:
  t_ = wb.worksheet("t_corpus_head")
  wb.del_worksheet(t_)
except:
  print ("Onglet inexistant !")

wb.add_worksheet("t_corpus_head", 1, 1)
export_sheet = wb.worksheet("t_corpus_head")
set_with_dataframe(export_sheet, df_vocabulary)

In [136]:
from google.colab import output
output.disable_custom_widget_manager()

#Table Traduction Corpus IA

In [74]:
from deep_translator import GoogleTranslator
language="rou"
iso_code = iso_codes[language]
input="Bonjour"
translation = GoogleTranslator(source='auto', target=iso_code).translate(input)
translation

'Bun ziua'

In [85]:
from deep_translator import DeeplTranslator
text = 'Quand avez-vous pris vos médicaments ?'
translated = DeeplTranslator(api_key="ea9751e0-a036-d65e-14af-3169e96b11e4:fx", source="fr", target="hu", use_free_api=True).translate(text)
translated

'Mikor vette be a gyógyszereit?'

In [57]:
def translation_deepl (input, language, language_filter, iso_code, ai_code):
  from deep_translator import DeeplTranslator
  translation = input
  state = 0 #par défaut

  if language == language_filter:
    translation = ""
    try:
      # print (input + " : " + language)
      translation = DeeplTranslator(api_key="ea9751e0-a036-d65e-14af-3169e96b11e4:fx", source="fr", target=iso_code, use_free_api=True).translate(input)
      if translation != "":
        print (input + " : " + translation)
        state = 2 #found
    except Exception as e:
      translation = ""
      pass
  else:
    ai_code = 0

  return translation, state, ai_code

In [75]:
def translation_google (input, language, language_filter, iso_code, ai_code):
  from deep_translator import GoogleTranslator
  translation = input
  state = 0 #par défaut
  print (language + " "+ language_filter)
  if language == language_filter:
    translation = ""
    try:
      print (input + " : " + language + " " + iso_code)
      translation = GoogleTranslator(source='auto', target=iso_code).translate(input)
      if translation != "":
        print (input + " : " + translation)
        state = 2 #found
    except Exception as e:
      print (e)
      translation = ""
      pass
  else:
    ai_code = 0

  return translation, state, ai_code

In [59]:
def translation_pons (input, language, language_filter, iso_code, ai_code):

  translation = input
  state = 0 #par défaut

  if language == language_filter:
    translation = ""
    try:
      # print (input + " : " + language)
      translation = PonsTranslator(source='fr', target=iso_code).translate(input)
      if translation != "":
        print (input + " : " + translation)
        state = 2 #found
    except Exception as e:
      translation = ""
      pass
  else:
    ai_code = 0

  return translation, state, ai_code

In [60]:
def translation_txtai (input, language, language_filter, iso_code, ai_cod):

  translation = input
  state = 0 #par défaut

  if language == language_filter:
    translation = ""
    try:
      # print (input + " : " + language)
      translation = translate(input, iso_code)
      if translation != "":
        print (input + " : " + translation)
        state = 2 #found
    except Exception as e:
      translation = ""
      pass
  else:
    ai_code = 0

  return translation, state, ai_code

1. On charge la base des traductions

In [61]:
sheet = db_langdeck_url
wb  = gc.open_by_url(sheet)
t_corpus = wb.worksheet("t_corpus_nlp")
data_t_corpus = t_corpus.get_all_values()
# on recharge avec les nouvelles données
df_corpus_nlp =  pd.DataFrame(data_t_corpus[1:], columns=data_t_corpus[0]).drop_duplicates()  
df_corpus_nlp = df_corpus_nlp[["uid","hash","maitre","expression","language","translation","translation_pivot","translation_state","translation_ai_source","translation_ai"]]									

### Appel de l'IA Google (12)

In [90]:
def call_nlp_google (language, iso_code, vc, df_langdeck_ai_temp):
  from deep_translator import GoogleTranslator
  if len(df_langdeck_ai_temp) > 0:
    print ("Using google translate")
    df_langdeck_ai_temp[["translation_ai","translation_state","translation_ai_source"]] = df_langdeck_ai_temp[["translation","expression","language","translation_state"]].apply(
        lambda x:translation_google(x[1],x[2],language, iso_code, vc), 
        axis=1, 
        result_type="expand")
  df_google = df_langdeck_ai_temp.copy()   
  return df_google   

In [91]:
def call_nlp_deepl (language, iso_code, vc, df_langdeck_ai_temp):
  from deep_translator import DeeplTranslator
  if len(df_langdeck_ai_temp) > 0:
    print ("Using deepl translate")
    df_langdeck_ai_temp[["translation_ai","translation_state","translation_ai_source"]] = df_langdeck_ai_temp[["translation","expression","language","translation_state"]].apply(
        lambda x:translation_deepl(x[1],x[2], language, iso_code, vc), 
        axis=1, 
        result_type="expand")
  df_deepl = df_langdeck_ai_temp.copy()
  return df_deepl


In [64]:
df_langdeck_ai_temp = df_langdeck.copy()
df_langdeck_ai_temp.loc[(~df_langdeck_ai_temp["uid"].isin(df_corpus_nlp["uid"])) & (df_langdeck_ai_temp["language"]==iso_code)]

,hash,uid,maitre,ordre,expression,idx,timestamp,language,translation,language_fulltext,...,theme_3,theme_1_svg,theme_2_svg,theme_3_svg,unixtime,translation_state,translation_pivot,translation_ai_source,translation_score,translation_state_txt


In [96]:
def call_nlp_language_vendor (language, iso_code, vc):
  # dataframe temporaire pour chaque appel IA
  df_langdeck_ai_temp = df_langdeck.copy()
  # ajout d'un uid par concaténation des 3 clés
  df_langdeck_ai_temp["uid"] = df_langdeck_ai_temp["hash"].map(str) + vc 
  df_langdeck_ai_temp = df_langdeck_ai_temp [["uid","hash","maitre","expression","language","translation","translation_pivot","translation_state","translation_ai_source"]]
  df_langdeck_ai_temp["translation_ai"]=""

  # on exclut les entrées déjà renseignées par l'iA
  df_langdeck_ai_temp = df_langdeck_ai_temp.loc[(~df_langdeck_ai_temp["uid"].isin(df_corpus_nlp["uid"])) & (df_langdeck_ai_temp["language"]==language)]
  # on appelle la traduction IA si :
  # - le uid n'existe pas dans la DF IA df_corpus_nlp
  # - le filtre est positionné  
  #df_langdeck_ai_temp = df_langdeck_ai_temp.head()
  for dic in dict_vendor:
    if dic["vendor-code"]==vc:
      if dic["vendor-abbv"]=="google":
        print ("Appel de call_nlp_google"+ iso_code)
        df_google = call_nlp_google (language, iso_code, vc, df_langdeck_ai_temp)
        df_return = df_google
      elif dic["vendor-abbv"]=="deepl":
        print ("Appel de call_nlp_deepl" + iso_code)
        df_deepl = call_nlp_deepl (language, iso_code, vc, df_langdeck_ai_temp)
        df_return = df_deepl
  return df_return

In [98]:
df_corpus_nlp

,uid,hash,maitre,expression,language,translation,translation_pivot,translation_state,translation_ai_source,translation_ai
0,f9bc13efams12,f9bc13efams,Médicament,À quel moment de la journée vous prenez le méd...,ams,في أي وقت من اليوم تتناول الدواء؟,At what time of day do you take the medicine?,2,12,في أي وقت من اليوم تتناول الدواء؟
1,6e65dde7ams12,6e65dde7ams,Abcès,Abcès,ams,خُرّاج,Abscesses,2,12,الخراجات
2,80cbaceaams12,80cbaceaams,Abdomen,Abdomen,ams,البطن,Abdomen,2,12,البطن
3,95c4a863ams12,95c4a863ams,Fébrile,Accès fébrile concomitant chez un autre membre...,ams,نوبة حمى مزامنة عند فرد اخر من الأسرة,Concomitant febrile attack in an other family ...,2,12,هل يصاحب ذلك نوبة حمى في أحد أفراد الأسرة الآخ...
4,30579682ams12,30579682ams,Accident,Accident,ams,حادث,Accident,2,12,حادثة
...,...,...,...,...,...,...,...,...,...,...
1901,729ec172arm12,729ec172arm,Cholestérol,Cholestérol,arm,,,0,12,
1902,85353d17arm12,85353d17arm,Acouphènes,Acouphènes,arm,,,0,12,
1903,d286204carm12,d286204carm,Hormone,Hormone,arm,,,0,12,
1904,1b72e89barm12,1b72e89barm,Intolérance,Intolérance,arm,,,0,12,


In [99]:
try:
  t_data = wb.worksheet("t_corpus_nlp")
  wb.del_worksheet(t_data)
except:
  print ("Onglet inexistant !")

wb.add_worksheet("t_corpus_nlp", 1, 1)
export_sheet = wb.worksheet("t_corpus_nlp")
set_with_dataframe(export_sheet, df_corpus_nlp)

Onglet inexistant !


In [97]:
# concaténation des dataframes
frames =[]
frames.append(df_corpus_nlp)

for dic in dict_lang_vendor:
  language = dic["language"]
  iso_code = dic["iso-code"]
  print (iso_code)  
  filter = iso_code
  for vc in dic["vendor-code"]:
    print ("\t"+vc)
    df_return = call_nlp_language_vendor (language, iso_code, vc)
    print (len(df_return))
    frames.append(df_return)

# quand tout a été itéré, on concatène    
df_langdeck_ai = pd.concat(frames)
print (len(df_langdeck_ai))
# on uniformise le type
df_langdeck_ai['translation_ai_source'] = df_langdeck_ai['translation_ai_source'].astype(str)
# on supprime les doublons
df_langdeck_ai = df_langdeck_ai.drop_duplicates(subset=['uid'], keep='last')
# on supprime les codes ai == 0
df_langdeck_ai = df_langdeck_ai.loc[df_langdeck_ai["translation_ai_source"]!="0"]
# on enregistre dans GSheet
try:
  t_data = wb.worksheet("t_corpus_nlp")
  wb.del_worksheet(t_data)
except:
  print ("Onglet inexistant !")

wb.add_worksheet("t_corpus_nlp", 1, 1)
export_sheet = wb.worksheet("t_corpus_nlp")
set_with_dataframe(export_sheet, df_langdeck_ai)


ar
	12
Appel de call_nlp_googlear
Using google translate
ams ams
Lait : ams ar
Lait : ليت
ams ams
J’ai cassé mes lunettes : ams ar
J’ai cassé mes lunettes : كسرت نظارتي
ams ams
Menton : ams ar
Menton : ذقن
ams ams
Opticien : ams ar
Opticien : اخصائي نظارات
ams ams
Regardez droit devant vous! : ams ar
Regardez droit devant vous! : تطلع للمستقبل!
ams ams
Rétine : ams ar
Rétine : شبكية العين
ams ams
Avez-vous du cholestérol ? : ams ar
Avez-vous du cholestérol ? : هل تعاني من الكوليسترول؟
ams ams
Cholestérol : ams ar
Cholestérol : الكوليسترول
ams ams
Acouphènes : ams ar
Acouphènes : طنين
ams ams
Hormone : ams ar
Hormone : هرمون
ams ams
Intolérance : ams ar
Intolérance : تعصب
ams ams
Prothèse : ams ar
Prothèse : بدلة
ams ams
Êtes-vous suivi pour des soucis de santé particuliers ? : ams ar
Êtes-vous suivi pour des soucis de santé particuliers ? : هل تتم متابعتك لأسباب صحية معينة؟
ams ams
Intolérance à un médicament : ams ar
Intolérance à un médicament : عدم تحمل المخدرات
14
ru
	12
Appel de c

KeyboardInterrupt: ignored

In [95]:
df_langdeck_ai

,uid,hash,maitre,expression,language,translation,translation_pivot,translation_state,translation_ai_source,translation_ai
0,f9bc13efams12,f9bc13efams,Médicament,À quel moment de la journée vous prenez le méd...,ams,في أي وقت من اليوم تتناول الدواء؟,At what time of day do you take the medicine?,2,12,في أي وقت من اليوم تتناول الدواء؟
1,6e65dde7ams12,6e65dde7ams,Abcès,Abcès,ams,خُرّاج,Abscesses,2,12,الخراجات
2,80cbaceaams12,80cbaceaams,Abdomen,Abdomen,ams,البطن,Abdomen,2,12,البطن
3,95c4a863ams12,95c4a863ams,Fébrile,Accès fébrile concomitant chez un autre membre...,ams,نوبة حمى مزامنة عند فرد اخر من الأسرة,Concomitant febrile attack in an other family ...,2,12,هل يصاحب ذلك نوبة حمى في أحد أفراد الأسرة الآخ...
4,30579682ams12,30579682ams,Accident,Accident,ams,حادث,Accident,2,12,حادثة
...,...,...,...,...,...,...,...,...,...,...
2046,f9bc13efhun14,f9bc13efhun,Médicament,À quel moment de la journée vous prenez le méd...,hun,Mikor veszi be a gyógyszert?,À quel moment de la journée vous prenez le méd...,2,14,A nap melyik szakában veszi be a gyógyszert?
2047,6e65dde7hun14,6e65dde7hun,Abcès,Abcès,hun,Tályogok,Abcès,2,14,Tályog
2048,80cbaceahun14,80cbaceahun,Abdomen,Abdomen,hun,Has,Abdomen,2,14,Has
2049,95c4a863hun14,95c4a863hun,Fébrile,Accès fébrile concomitant chez un autre membre...,hun,Lázas valaki a családban ?,Accès fébrile concomitant chez un autre membre...,2,14,Egyidejű lázas roham egy másik családtagnál


In [93]:
# concaténation des dataframes
frames =[]
frames.append(df_corpus_nlp)
frames.append(df_return)
#frames.append(df_deepl)
df_langdeck_ai = pd.concat(frames)
# on uniformise le type
df_langdeck_ai['translation_ai_source'] = df_langdeck_ai['translation_ai_source'].astype(str)
# on supprime les doublons
df_langdeck_ai = df_langdeck_ai.drop_duplicates(subset=['uid'], keep='last')
# on supprime les codes ai == 0
df_langdeck_ai = df_langdeck_ai.loc[df_langdeck_ai["translation_ai_source"]!="0"]


Onglet inexistant !


APIError: ignored

In [ ]:
# dataframe temporaire pour chaque appel IA
ai = "12"
df_langdeck_ai_temp = df_langdeck.copy()

# ajout d'un uid par concaténation des 3 clés
df_langdeck_ai_temp["uid"] = df_langdeck_ai_temp["hash"].map(str) + ai 
df_langdeck_ai_temp = df_langdeck_ai_temp [["uid","hash","maitre","expression","language","translation","translation_pivot","translation_state","translation_ai_source"]]
df_langdeck_ai_temp["translation_ai"]=""

# on exclut les entrées déjà renseignées par l'iA
df_langdeck_ai_temp = df_langdeck_ai_temp.loc[(~df_langdeck_ai_temp["uid"].isin(df_corpus_nlp["uid"])) & (df_langdeck_ai_temp["language"]==filter)]
# on appelle la traduction IA si :
# - le uid n'existe pas dans la DF IA df_corpus_nlp
# - le filtre est positionné


In [ ]:
df_langdeck_ai_temp

In [ ]:
# on appelle la traduction IA si :
# - le uid n'existe pas dans la DF IA df_corpus_nlp
# - le filtre est positionné
if len(df_langdeck_ai_temp) > 0:
  from deep_translator import GoogleTranslator
  df_langdeck_ai_temp[["translation_ai","translation_state","translation_ai_source"]] = df_langdeck_ai_temp[["translation","translation_pivot","language","translation_state"]].apply(
      lambda x:translation_google(x[1],x[2], filter, ai), 
      axis=1, 
      result_type="expand")
df_google = df_langdeck_ai_temp.copy()

## Appel DeepL

In [ ]:
# dataframe temporaire pour chaque appel IA
df_langdeck_ai_temp = df_langdeck.copy()
ai = "14"
# ajout d'un uid par concaténation des 3 clés
df_langdeck_ai_temp["uid"] = df_langdeck_ai_temp["hash"].map(str) + ai 
df_langdeck_ai_temp = df_langdeck_ai_temp [["uid","hash","maitre","expression","language","translation","translation_pivot","translation_state","translation_ai_source"]]
df_langdeck_ai_temp["translation_ai"]=""

# on exclut les entrées déjà renseignées par l'iA
df_langdeck_ai_temp = df_langdeck_ai_temp.loc[(~df_langdeck_ai_temp["uid"].isin(df_corpus_nlp["uid"])) & (df_langdeck_ai_temp["language"]==filter)]
# on appelle la traduction IA si :
# - le uid n'existe pas dans la DF IA df_corpus_nlp
# - le filtre est positionné


In [ ]:
df_langdeck_ai_temp

In [ ]:
from deep_translator import DeeplTranslator
df_langdeck_ai_temp[["translation_ai","translation_state","translation_ai_source"]] = df_langdeck_ai_temp[["translation","translation_pivot","language","translation_state"]].apply(
    lambda x:translation_deepl(x[1],x[2], filter, ai), 
    axis=1, 
    result_type="expand")
df_deepl = df_langdeck_ai_temp.copy()

## On sauvegarde

# FIN IA

In [ ]:
from deep_translator import PonsTranslator
filter = "hun"
ai = 13
df_langdeck_ai_temp[["translation_ai","translation_state","translation_ai_source"]] = df_langdeck_ai_temp[["translation","translation_pivot","language","translation_state"]].apply(
    lambda x:translation_pons(x[1],x[2], filter, ai), 
    axis=1, 
    result_type="expand")
df_pons = df_langdeck_ai_temp.copy()

### Ajout du dataset Google dans le dataset global IA

In [ ]:
frames =[]
frames.append(df_google)
frames.append(df_pons)
df_langdeck_ai = pd.concat(frames)

In [ ]:
# ajout d'un uid par concaténation des 3 clés
df_langdeck_ai["uid"] = df_langdeck_ai["hash"].map(str) + df_langdeck_ai["translation_ai_source"].map(str) 

In [ ]:
df_langdeck_ai.loc[(df_langdeck_ai["language"]=="hun") & (df_langdeck_ai["hash"]=="30579682hun")]

NameError: ignored

In [ ]:
def fetch_deep_translator(hashid, language, vendor-name_code, row):
  iso_code = iso_codes[language]
  ai_dict = next(filter(lambda x: x.get('vendor-code') == vendor-name_code,deep_translator_vendor-name_codes))
  vendor-name = ai_dict["vendor-name"]
  pivot = ai_dict["pivot"]
  if pivot == "en":
    pivot_expr = row["translation_pivot"]
  else:
    pivot_expr = row["expression"]
  message = "Appel de l'IA Deep " + vendor-name + " pour la langue " + iso_code + " pour l'expression " + pivot_expr


  df_langdeck_ai_temp[["translation_ai","translation_state","translation_ai_source"]] = df_langdeck_ai_temp[["translation","translation_pivot","language","translation_state"]].apply(
      lambda x:translation_google(x[1],x[2], filter, ai), 
      axis=1, 
      result_type="expand")

  return message

In [ ]:
hash_ai = "7c15835bukr12"
len(df_corpus_nlp.loc[df_corpus_nlp["hash_ai"]==hash_ai])

0

In [ ]:
filtre = "ams"
vc = "11"
for key, value in df_langdeck.iterrows():
  if value["language"]==filtre:
    # on recherche la clé hash+ia dana la table deep
    hash_ai = value["hash"]+vc
    found = df_corpus_nlp.loc[df_corpus_nlp["hash_ai"]==hash_ai]
    # pas trouvé, on passe la ligne pour appel ia
    if len(found)==0:
      print ("Recherche de " +hash_ai)
      print (fetch_deep_translator(value["hash"], value["language"], vc, value))


In [ ]:

translated = GoogleTranslator(source='auto', target='de').translate("keep it up, you are awesome")  # output -> Weiter so, du bist großartig

NameError: ignored

Google Translator

Pons

In [ ]:
'''
filter = "rou"
ai = 11
# algorithme :
# si state==1(pending) ou 2(validé) => pas d'appel IA
# si state==0 => appel ia
df_langdeck[["translation","translation_state","translation_ai_source"]] = df_langdeck[["translation","translation_pivot","language","translation_state"]].apply(
    lambda x:fetch_deep_tlr_txtai(x[1],x[2], filter, ai) 
    if x[3]==0 
    else (x[0],x[3],0), 
    axis=1, 
    result_type="expand")
'''

'\nfilter = "rou"\nai = 11\n# algorithme :\n# si state==1(pending) ou 2(validé) => pas d\'appel IA\n# si state==0 => appel ia\ndf_langdeck[["translation","translation_state","translation_ai_source"]] = df_langdeck[["translation","translation_pivot","language","translation_state"]].apply(\n    lambda x:fetch_translation_ia_txtai(x[1],x[2], filter, ai) \n    if x[3]==0 \n    else (x[0],x[3],0), \n    axis=1, \n    result_type="expand")\n'

In [ ]:
df_langdeck_ai_temp["hash_ai"]=df_langdeck_ai_temp[["hash","translation_ai_source"]].apply(lambda x:x[0]+str(x[1]), axis=1)

In [ ]:
df_langdeck_ai_temp = df_langdeck_ai_temp.drop(df_langdeck_ai_temp[df_langdeck_ai_temp["translation_ai_source"]==0].index)

In [ ]:
frames = [df_langdeck_ai, df_langdeck_ai_temp]
df_langdeck_ai = pd.concat(frames)

In [ ]:
df_langdeck_ai.loc[df_langdeck_ai["translation_ai_source"]==13]

,hash,maitre,expression,language,translation,translation_pivot,translation_state,translation_ai_source,translation_ai,hash_ai
0,f9bc13efams,Médicament,À quel moment de la journée vous prenez le méd...,ams,في أي وقت من اليوم تتناول الدواء؟,At what time of day do you take the medicine?,2,13,أثْناءَ,f9bc13efams13
1,6e65dde7ams,Abcès,Abcès,ams,خُرّاج,Abscesses,2,13,خُرَاج <خُرَاجات>,6e65dde7ams13
2,80cbaceaams,Abdomen,Abdomen,ams,البطن,Abdomen,2,13,البَطْن,80cbaceaams13
3,95c4a863ams,Fébrile,Accès fébrile concomitant chez un autre membre...,ams,نوبة حمى مزامنة عند فرد اخر من الأسرة,Concomitant febrile attack in an other family ...,0,13,,95c4a863ams13
4,30579682ams,Accident,Accident,ams,حادث,Accident,2,13,مُصَادَفَة,30579682ams13
...,...,...,...,...,...,...,...,...,...,...
5529,e352c9c7ams,Sinusite,Sinusite,ams,أضف تعليقًا,Sine [K],2,13,جيب الزاية,e352c9c7ams13
5530,be431a55ams,Ulcère,Ulcère,ams,Ulcer,Ulcer,2,13,قَرْحَة <قَرْحات>,be431a55ams13
5531,7019983eams,Ulcère,Ulcère de l'estomac,ams,مُلْسَكَر,Stomach ulcer,2,13,قَرْحَةُ المَعِدَة,7019983eams13
5532,a2e70804ams,Ulcère,Ulcère de l'intestin grêle,ams,أُجُزأة صغيرة من الأم,Ulcer of the small intestine,2,13,المِعَى الدَّقيق,a2e70804ams13


In [ ]:
try:
  t_corpus_all_entries = wb.worksheet("t_translate_deep")
  wb.del_worksheet(t_corpus_all_entries)
except:
  print ("Onglet inexistant !")

wb.add_worksheet("t_translate_deep", 1, 1)
export_sheet = wb.worksheet("t_translate_deep")
set_with_dataframe(export_sheet, df_langdeck_ai)

Onglet inexistant !


In [ ]:

#df_langdeck["translation_ai_source"] = np.nan
df_langdeck["translation_score"] = df_langdeck["translation_state"].apply(lambda x: (1.0 if x==1 else 0.5 if x==2 else 0.0))
df_langdeck["translation_state_txt"] = df_langdeck["translation_state"].apply(lambda x: ("OK" if x==1 else "Pending" if x==2 else "Reject"))

In [ ]:
try:
  t_corpus_all_entries = wb.worksheet("t_corpus_all_entries")
  wb.del_worksheet(t_corpus_all_entries)
except:
  print ("Onglet inexistant !")

wb.add_worksheet("t_corpus_all_entries", 1, 1)
export_sheet = wb.worksheet("t_corpus_all_entries")
set_with_dataframe(export_sheet, df_langdeck)

# Vue Lexique (deprecated)

In [ ]:
t_lexique = wb.worksheet("t_lexicon")
# on recharge avec les nouvelles données

### nouvelles colonnes Thèmes

In [ ]:
# on crée une liste des valeurs uniques des 3 colonnes de thèmes
frames = [df_langdeck["theme_1"],df_langdeck["theme_2"],df_langdeck["theme_3"]]
vk_theme = list(set(pd.concat(frames).to_list()))
vk_theme.remove('')

In [ ]:
# on ajoute les colonnes à partir de la liste et on valorise T/F
df_lexique = df_langdeck[["hash","maitre","ordre","expression","language","language_fulltext","theme_1","theme_2","theme_3"]].copy()
df_lexique = pd.concat([df_lexique, pd.DataFrame(columns = vk_theme)])
for item in vk_theme:
  df_lexique[item] = df_lexique[["theme_1","theme_2","theme_3"]].apply(lambda x:True if (x[0]==item or x[1]==item or x[2]==item) else False, axis=1)

In [ ]:
frames=[]
df=pd.DataFrame()
frames.append(df)
# pour chaque thème on ajoute les expressions trouvées
for item in vk_theme:
  df_temp = df_lexique.loc[df_lexique[item] == True]
  df_temp.insert(0,'theme', item)
  df_temp.insert(0,'section', df_temp["ordre"].apply(lambda x:("Vocabulaire" if x==1 else "Expression" if x==2 else "Phrase" if x>=3 else "NONE")))
  df_temp.insert(0,'theme_hash', df_temp["hash"].apply(lambda x:"".join(item.split())+x))
  frames.append(df_temp)
  # on concatène le tout dans un new dataframe
  df_langdeck_theme = pd.concat(frames)

In [ ]:
len(df_langdeck_theme)

7280

In [ ]:
df_langdeck_theme.iloc[1]

theme_hash                  Médecinegénérale6e65dde7ams
section                                     Vocabulaire
theme                                 Médecine générale
hash                                        6e65dde7ams
maitre                                            Abcès
ordre                                               1.0
expression                                        Abcès
language                                            ams
language_fulltext                                 Arabe
theme_1                               Médecine générale
theme_2                                                
theme_3                                                
Médecine générale                                  True
Etat civil                                        False
Vaccination                                       False
Hépato-Gastroentérologie                          False
Douleur                                           False
Nourriture                                      

In [ ]:
try:
  tbl = wb.worksheet("t_voc_synth")
  wb.del_worksheet(tbl)
except:
  print ("Onglet inexistant !")

wb.add_worksheet("t_voc_synth", 1, 1)
export_sheet = wb.worksheet("t_voc_synth")
set_with_dataframe(export_sheet, df_langdeck)

In [ ]:
try:
  t_lexique = wb.worksheet("t_lexicon")
  wb.del_worksheet(t_lexique)
except:
  print ("Onglet inexistant !")

wb.add_worksheet("t_lexicon", 1, 1)
export_sheet = wb.worksheet("t_lexicon")
set_with_dataframe(export_sheet, df_langdeck_theme)

# FIN pour l'instant

# Appel IA Google Translate

In [ ]:
# txtAI
translation = translate("Forearm", "ar")
translation

'لـ مُرْرْم'

In [ ]:
def txtai_alpha (input, iso_code):
  try:
    translation = translate(input, "ar")
  except Exception as e:
    translation = ""
    pass
  return translation, 2


In [ ]:
df_txtai = df_langdeck.loc[(df_langdeck["language"]=="ams") & (df_langdeck["translation_state"]==0) & (df_langdeck["idx"]=="B")]

In [ ]:
df_txtai

,hash,uid,maitre,ordre,expression,idx,timestamp,language,translation,language_fulltext,...,translation_state,Corps humain,Etat civil,Hépato-Gastroentérologie,Examen du patient,Vaccination,Maladie,Dossier du patient,Douleur,Médecine générale
56,f3bff76cams,f3bff76c,Bonjour,1.0,Bonjour,B,2022-03-31 14:33:26,ams,,Arabe,...,0.0,False,False,False,False,False,False,True,False,True
58,b80ffb21ams,b80ffb21,Bouche,1.0,Bouche,B,2022-03-31 15:25:18,ams,,Arabe,...,0.0,True,False,False,False,False,False,False,False,False


In [ ]:
df_txtai[["translation","translation_state"]]=df_txtai[["expression","translation_state"]].apply(lambda x: txtai_alpha(x[0], "ar") if x[1]==0 else "", axis=1, result_type="expand")

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:3641: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [ ]:
df_txtai

,hash,uid,maitre,ordre,expression,idx,timestamp,language,translation,language_fulltext,...,translation_state,Corps humain,Etat civil,Hépato-Gastroentérologie,Examen du patient,Vaccination,Maladie,Dossier du patient,Douleur,Médecine générale
56,f3bff76cams,f3bff76c,Bonjour,1.0,Bonjour,B,2022-03-31 14:33:26,ams,مرحباً، مرحباً، مرحباً، مرحباً، مرحباً، مرحباً...,Arabe,...,2,False,False,False,False,False,False,True,False,True
58,b80ffb21ams,b80ffb21,Bouche,1.0,Bouche,B,2022-03-31 15:25:18,ams,لاشتششششششششششششش,Arabe,...,2,True,False,False,False,False,False,False,False,False


## Mise à jour inutile dans Langdeck !!

In [ ]:
df_airtable_for_langdeck = get_df_airtable_for_langdeck()
df_airtable_for_langdeck["last_updated"] = df_airtable_for_langdeck[["hash","unixtime"]].apply(lambda x:compare_langdeck_time(x[0],x[1]), axis=1) 
df_newly_updated_in_airtable = df_airtable_for_langdeck.loc[df_airtable_for_langdeck["last_updated"]]


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: FutureWarning: casting datetime64[ns] values to int64 with .astype(...) is deprecated and will raise in a future version. Use .view(...) instead.
  from ipykernel import kernelapp as app


### Récupération des audios

In [ ]:
df_newly_updated_in_airtable


,uid,maitre,ordre,expression,idx,timestamp,language,translation,hash,language_fulltext,unixtime,last_updated
78,8745562b,Comment,3,Comment allez-vous ?,C,2022-04-11 13:41:03,ams,NaN,8745562bams,Arabe,1649684463000000000,True
187,2bd46073,Main,1,Main,M,2022-04-11 13:41:01,ams,NaN,2bd46073ams,Arabe,1649684461000000000,True
201,80e0c323,Médicament,1,Médicament(s),M,2022-04-11 13:41:02,ams,تَورُّم,80e0c323ams,Arabe,1649684462000000000,True
257,ba312208,Âge,3,Quel âge avez-vous ?,A,2022-04-11 13:41:02,ams,ما سبب زيارتك؟,ba312208ams,Arabe,1649684462000000000,True
529,2bd46073,Main,1,Main,M,2022-04-11 13:41:01,eng,Hand,2bd46073eng,Anglais,1649684461000000000,True
...,...,...,...,...,...,...,...,...,...,...,...,...
5045,ba312208,Âge,3,Quel âge avez-vous ?,A,2022-04-11 13:41:02,pol,Ile masz lat ?,ba312208pol,Polonais,1649684462000000000,True
5208,8745562b,Comment,3,Comment allez-vous ?,C,2022-04-11 13:41:03,tig,NaN,8745562btig,Tigrinya,1649684463000000000,True
5317,2bd46073,Main,1,Main,M,2022-04-11 13:41:01,tig,NaN,2bd46073tig,Tigrinya,1649684461000000000,True
5331,80e0c323,Médicament,1,Médicament(s),M,2022-04-11 13:41:02,tig,ከኒና,80e0c323tig,Tigrinya,1649684462000000000,True


## Mise à jour dans Langdeck

In [ ]:
l = len(df_newly_updated_in_airtable)
# on parcoure la table cible
for i in range (2, l):
  # un enregistrement
  tuple_list = t_corpus.row_values(i)
  # on recherche la clé dans le dataframe source
  t_ = df_airtable_for_langdeck.loc[(df_airtable_for_langdeck['uid']==tuple_list[0]) & (df_airtable_for_langdeck['language']==tuple_list[6])]
  # verification du hash
  if len(t_)==1:
    t_corpus.update_cell(i, 9, t_["hash"].values[0])
  # comparaison des ts (A=AT;B=GG)
  ts_a = pd.to_datetime(t_["timestamp"].values[0], "%Y-%m-%d %H:%M:%S")
  ts_b = datetime.datetime.strptime(tuple_list[5], "%Y-%m-%d %H:%M:%S")
  # print (t_["hash"].values[0] + ": Airtable ts " + str(ts_a) + " GG ts " + str(ts_b))
  if ts_a > ts_b:
    # cas Airtable + récent => update dans Langdeck
    t_corpus.update_cell(i, 2, t_["maitre"].values[0])
    t_corpus.update_cell(i, 3, int(t_["ordre"].values[0]))
    t_corpus.update_cell(i, 4, t_["expression"].values[0])
    t_corpus.update_cell(i, 5, t_["idx"].values[0])
    if not pd.isnull(t_["translation"].values[0]):
      t_corpus.update_cell(i, 8, t_["translation"].values[0])    
  elif ts_b > ts_a:
    # cas Langdeck + récent => update translation dans AT uniqumeent !
    record = airtable.match('UID', tuple_list[0])
    if bool(record) is True:
      if not pd.isnull(tuple_list[7]):
        aDict = {}
        aDict[tuple_list[6]] = tuple_list[7]
        airtable.update(record['id'], aDict)
        print (record['id'])


    


In [ ]:
frames = [df_airtable_for_langdeck, df_langdeck]
result = pd.concat(frames, keys=["hash"])
result.drop_duplicates(subset=['hash'], inplace=True, keep='last')

In [ ]:
result

uid      maitre  ordre                                         expression idx           timestamp language                                        translation         hash language_fulltext
hash 0     f9bc13ef  Médicament      3  À quel moment de la journée vous prenez le méd...   M 2022-04-01 08:15:21      ams                  في أي وقت من اليوم تتناول الدواء؟  f9bc13efams             Arabe
     1     6e65dde7       Abcès      1                                              Abcès   A 2022-03-29 15:58:04      ams                                             خُرّاج  6e65dde7ams             Arabe
     2     80cbacea     Abdomen      1                                            Abdomen   A 2022-04-07 08:42:16      ams                                              البطن  80cbaceaams             Arabe
     3     95c4a863     Fébrile      3  Accès fébrile concomitant chez un autre membre...   F 2022-04-01 08:27:19      ams              نوبة حمى مزامنة عند فرد اخر من الأسرة  95c4a863ams             Arabe
     4     30579682    Accident      1                                           Accident   A 2022-03-29 15:58:05      ams                                               حادث  30579682ams             Arabe
...             ...         ...    ...                                                ...  ..                 ...      ...                                                ...          ...               ...
     5451  d112f379       Poids      3                        Votre poids est-il stable ?   P 2022-03-31 14:33:52      tig                                                NaN  d112f379tig          Tigrinya
     5452  2344ba84   Accoucher      3  Vous avez accouché par césarienne ou voie basse ?   A 2022-03-29 16:00:33      tig                             ብኦፕረሽን ወይ ብኖርማር ወሊድኪ ?  2344ba84tig          Tigrinya
     5453  36e66161  Contagieux      3            Y a-t-il des cas contagieux à l'école ?   C 2022-03-29 16:00:33      tig                                 አብ ትምህርቲ ዲዩ ለጊብካ ?  36e66161tig          Tigrinya
     5454  c32f5fd9   Chronique      3  Y a-t-il des maladies chroniques dans votre fa...   C 2022-04-01 08:23:46      tig                      ካብ በተሰብካ ሕዱር ሕማሕ ዘለዎ አለካ ዲዩ ?  c32f5fd9tig          Tigrinya
     5455  d3dba8e4        Zona      1                                               Zona   Z 2022-03-29 16:00:34      tig  ዞና ዝብሃል ሕማም ኣብ ቖርበት ማለት ብደገ ዝቖስል ብርቱዕ ቓንዛ ዘለዎ ...  d3dba8e4tig          Tigrinya

[5456 rows x 10 columns]

In [ ]:
df_airtable_for_langdeck[(df_airtable_for_langdeck['hash'].isin(df_langdeck['hash']) & ~df_airtable_for_langdeck['timestamp'].isin(df_langdeck['timestamp']) )]

,uid,maitre,ordre,expression,idx,timestamp,language,translation,hash,language_fulltext
2,80cbacea,Abdomen,1,Abdomen,A,2022-04-07 08:42:16,ams,البطن,80cbaceaams,Arabe


In [ ]:
set_with_dataframe(t_corpus, result)

END

### Vocabulaire avec groupement :


1.   Index alpha
2.   Mot clé
3.   Ordre



In [ ]:
vk_voc = airtable.get_all(view='Vocabulary', sort=['idx_cor','Maitre','Ordre'])
df_voc = pd.DataFrame.from_records((r['fields'] for r in vk_voc))
df_voc = df_voc[['idx_cor','Maitre','Ordre','Proposition','CATEGRAM','TYPE','Thématique']]

### Traductions avec groupement :


1.   Index alpha
2.   Mot clé
3.   Ordre

In [ ]:
df_voc_trad = airtable.get_all(view='Traductions', sort=['idx_cor','Maitre','Ordre'])
df_voc_trad = pd.DataFrame.from_records((r['fields'] for r in df_voc_trad))
df_voc_trad = df_voc_trad[['idx_cor','Maitre','Ordre','Proposition','CATEGRAM','TYPE','Thématique',
             'eng','ams','tur','rus','ukr','rou','hun','tig','alb','geo','arm',
             'dar','pst','esp','pol']]

In [ ]:
# Corpus_consolidation
uri = "https://docs.google.com/spreadsheets/d/1cSnSnnQunL-I7hpyYkIqbtuGr55unVJ3Qe8URSrjTr8/edit?usp=sharing"

wb = get_CCDB_wb(uri)
db_cc = get_CCDB_data (wb, 0)

nom_onglet = 'Lexique'
# si l'onglet existe déjà
try:
  ws = wb.worksheet(nom_onglet)
  wb.del_worksheet(ws)
except:
  print ("Onglet inexistant !")

wb.add_worksheet(nom_onglet, 1, 1)
export_sheet = wb.worksheet(nom_onglet)
set_with_dataframe(export_sheet, df_voc_trad)

Onglet inexistant !


In [ ]:
grouped = df_voc.groupby(['idx_cor','Maitre'])
'''
for name, group in grouped:
  print (name)
  print (group["Proposition"])
'''

'\nfor name, group in grouped:\n  print (name)\n  print (group["Proposition"])\n'

Recherche des UID manquants dans Airtable

In [ ]:
def airtable_get_missing_uids(df):
  return df.loc[pd.isna(df["UID"])]


Recherche dans Airtable par Séquence

In [ ]:
def airtable_get_records_by_sequence(df, sequence):
  return df.loc[df["Séquence"]==sequence]


In [ ]:
#df_vkat.loc[df_vkat["Séquence"]=="1. Présentation + antécédents"]

In [ ]:
airtable_get_missing_uids(df_vkat)

,Proposition,CATEGRAM,DEPENDANCE,idx_cor,TYPE,UID,VOCABULAIRE,Thématique,idx,Ordre,...,Updates,Séquence,MedGen,Statut,Fiche,Phrase Assoc,Définition,Corpus copy,DOMAINE,Exp Assoc
6,~ chirurgicale,[exp],[reco8iFRctnJgMJCl],I,expression,NaN,NaN,[Vaccination],I,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26,Accouchement,[nm],NaN,A,vocabulaire,NaN,NaN,[Médecine générale],NaN,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
27,Accoucher,[vi],NaN,A,vocabulaire,NaN,NaN,[Médecine générale],NaN,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
40,Appareil ~,[exp],NaN,D,expression,NaN,NaN,[Médecine générale],NaN,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
41,Assurance,[nf],NaN,A,vocabulaire,NaN,Général,[Médecine générale],NaN,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
304,Urine,[nf],NaN,U,vocabulaire,NaN,NaN,NaN,NaN,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
306,Vaccinal,[adj],NaN,V,vocabulaire,NaN,NaN,NaN,NaN,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
308,"Vacciné, -e",[adj],NaN,V,vocabulaire,NaN,NaN,NaN,NaN,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
317,Votre poids a-t-il augmenté ?,[phrase],[recnFsFSoE1DCjYqt],P,phrase,NaN,NaN,[Médecine générale],NaN,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Rceherche des termes présents dans Central, présents dans AIRTBL mais sans UID

In [ ]:
vk_at_trad = airtable.get_all(view='Traductions',sort='Proposition')
df_vk_at_trad = pd.DataFrame.from_records((r['fields'] for r in vk_at_trad))
df_vk_at_trad = df_vk_at_trad[["Proposition","idx_cor","UID"]]
df_vk_at_trad_notna = df_vk_at_trad.loc[pd.notna(df_vk_at_trad["UID"])]
df_vk_at_trad_na = df_vk_at_trad.loc[pd.isna(df_vk_at_trad["UID"])]

### merge côté AT : avec les UID présents 

In [ ]:
df_merged = pd.merge(df_cc_global, df_vk_at_trad_notna, how="right", left_on='uid', right_on='UID')

## On complète avec les traductions

In [ ]:
for index, row in df_merged.iterrows():
  if row["uid"] is not None:
    record = airtable.match('UID', row["uid"])
    aDict = {}
    for i in vk_languages:
      key = i["trigramme"]
      aDict[key] = row[key]
      #print (record['id'] +" " + row["eng"])
    airtable.update(record['id'], aDict)


## On complète avec les UID manquants

In [ ]:
df_vk_at_trad_na

In [ ]:
df_merged = pd.merge(df_cc_global, df_vk_at_trad_na, how="right", left_on='expression', right_on='Proposition')

In [ ]:
df_merged = df_merged.loc[pd.notna(df_merged["uid"])]

In [ ]:
for index, row in df_merged.iterrows():
  if row["uid"] is not None:
    record = airtable.match('Proposition', row["expression"])
    aDict = {}
    key = "UID"
    aDict[key] = row["uid"]
    for i in vk_languages:
      key = i["trigramme"]
      aDict[key] = row[key]
      #print (record['id'] +" " + row["eng"])
    airtable.update(record['id'], aDict)

## On vide toute la base AT dans Central Consolidation

In [ ]:
vk_at = airtable.get_all(view='Admin',sort='Proposition')
df_vkat = pd.DataFrame.from_records((r['fields'] for r in vk_at))

In [ ]:
# Corpus_consolidation
uri = "https://docs.google.com/spreadsheets/d/1cSnSnnQunL-I7hpyYkIqbtuGr55unVJ3Qe8URSrjTr8/edit?usp=sharing"

wb = get_CCDB_wb(uri)
db_cc = get_CCDB_data (wb, 0)

nom_onglet = 'Termes'
# si l'onglet existe déjà
try:
  ws = wb.worksheet(nom_onglet)
  wb.del_worksheet(ws)
except:
  print ("Onglet inexistant !")

wb.add_worksheet(nom_onglet, 1, 1)
export_sheet = wb.worksheet(nom_onglet)
set_with_dataframe(export_sheet, df_vkat)



END

In [ ]:
record = airtable.match('UID', '30579682')
fields = {'eng': 'Accident'}
airtable.update(record['id'], fields)

{'createdTime': '2021-12-10T08:47:48.000Z',
 'fields': {'CATEGRAM': ['nm'],
  'Name ID': 'recGaRQGjV5qN3QBN',
  'Proposition': 'Accident',
  'TYPE': 'vocabulaire',
  'Thématique': 'Vaccination',
  'UID': '30579682',
  'VOCABULAIRE': 'Général',
  'eng': 'Accident',
  'idx': 'A',
  'idx_cor': 'A'},
 'id': 'recGaRQGjV5qN3QBN'}

In [ ]:
record

{'createdTime': '2021-12-10T08:47:48.000Z',
 'fields': {'CATEGRAM': ['nm'],
  'Name ID': 'recGaRQGjV5qN3QBN',
  'Proposition': 'Accident',
  'TYPE': 'vocabulaire',
  'Thématique': 'Vaccination',
  'UID': '30579682',
  'VOCABULAIRE': 'Général',
  'idx': 'A',
  'idx_cor': 'A'},
 'id': 'recGaRQGjV5qN3QBN'}

In [ ]:
def airtable_table_get(uri, params, headers):
  airtable_records = []
  run = True
  while run is True:
    response = requests.get(uri, params=params, headers=headers)
    airtable_response = response.json()
    airtable_records += (airtable_response['records'])
    if 'offset' in airtable_response:
      run = True
      params = (('offset', airtable_response['offset']),)
    else:
      run = False
  return airtable_records

# Table Corpus

In [ ]:

url = "https://api.airtable.com/v0/" + base_id + "/" + table_name


### On lit tous les enregistrements, puis on met dans un dataframe

# Nouvelle section

In [ ]:
airtable_records = airtable_table_get(url, params, headers)

airtable_rows = [] 
airtable_index = []
for record in airtable_records:
  airtable_rows.append(record['fields'])
  airtable_index.append(record['id'])
df = pd.DataFrame(airtable_rows, index=airtable_index)
df = df [['Proposition','UID', 'Name ID', 'DEPENDANCE', 'idx', 'idx_cor','Statut','Thématique']]
df.sort_values(by=['Proposition'], ascending=True, inplace=True)

In [ ]:
df.head(10)

,Proposition,UID,Name ID,DEPENDANCE,idx,idx_cor,Statut,Thématique
recKbPHqDTXNPwYK0,A jeun,f397cda1,recKbPHqDTXNPwYK0,NaN,J,J,NaN,Vaccination
rec1qTnfWxjhx6SBl,AME - Aide médicale d'État,f49563a9,rec1qTnfWxjhx6SBl,NaN,A,A,NaN,Vaccination
rec0HgtD22F7TWM3W,Abcès,6e65dde7,rec0HgtD22F7TWM3W,NaN,A,A,NaN,Médecine générale
recGaRQGjV5qN3QBN,Accident,30579682,recGaRQGjV5qN3QBN,NaN,A,A,NaN,Vaccination
rec3MxDq67MC982ng,Accident cérébral,6f214e4c,rec3MxDq67MC982ng,[recGaRQGjV5qN3QBN],A,A,NaN,Vaccination
reciexfGbNKvXzQg8,Alcool,NaN,reciexfGbNKvXzQg8,NaN,A,A,NaN,Médecine générale
recfY4uKkgK8Jp8GO,Allergie,8b195cf4,recfY4uKkgK8Jp8GO,NaN,A,A,NaN,Vaccination
recSKSrXVLhDfMq6w,Allergie alimentaire,f754eebf,recSKSrXVLhDfMq6w,[recfY4uKkgK8Jp8GO],A,A,NaN,Vaccination
rectthHdPchXi1zr9,Allergie médicamenteuse,7256c169,rectthHdPchXi1zr9,"[recfY4uKkgK8Jp8GO, recOivQOmeragygVr]",A,A,NaN,Vaccination
recHNzXfvWWm0Lwp8,Allergie à un médicament,316fa7ab,recHNzXfvWWm0Lwp8,"[recfY4uKkgK8Jp8GO, recOivQOmeragygVr]",A,A,NaN,Vaccination


In [ ]:
def lookup_uid (uid):
  try:
    record = df_cc_fr.loc[df_cc_fr["uid"]==uid]["index"].item()
  except Exception as e:
    list_no_uid.append(uid)
    return uid
  if record is not None:
    return record

In [ ]:
x = df_cc_fr.loc[df_cc_fr["uid"]=="72f0e8db"]["index"].item()

In [ ]:
list_no_uid = [] # uid non présents côté central = mots à créer
df['UID'] = df['UID'].apply(lambda x: make_unique_id() if pd.isna(x) else x)
df.idx_cor = df['UID'].apply(lambda x: lookup_uid(x))

convert the dataframe to a dictionary of dictionaries.

In [ ]:
df.tail()

,Proposition,UID,Name ID,DEPENDANCE,idx,idx_cor,Statut,Thématique
recia1L6nICYg6LJ8,Zona,d3dba8e4,recia1L6nICYg6LJ8,NaN,NaN,Z,NaN,Vaccination
recVvIHdNWf80zvsG,À quelle moment de la journée vous prenez le m...,0f86d1ef,recVvIHdNWf80zvsG,[recOivQOmeragygVr],M,0f86d1ef,Ramona,Médecine générale
recWp0rajm2qKcJuF,Êtes vous vacciné (e) ?,c5df267a,recWp0rajm2qKcJuF,NaN,NaN,Ê,NaN,Médecine générale
recBY5NXGM8UtBcaX,Êtes-vous enceinte ?,7768882c,recBY5NXGM8UtBcaX,NaN,NaN,Ê,NaN,Médecine générale
reczy8Os5Hc3qZ1yA,Œdème,72f0e8db,reczy8Os5Hc3qZ1yA,NaN,NaN,Œ,NaN,Médecine générale


In [ ]:
dict_records = df.tail().to_dict(orient="index")

In [ ]:
for record_id, upload_data in dict_records.items():
  record_url = url + "/" + record_id
  upload_dict = {"records" : [{"fields" : upload_data}]}
  upload_json = json.dumps(upload_dict)
  requests.post(url, data=upload_json, headers=headers)

In [ ]:
headers

{'Authorization': 'Bearer keyrI98TIqu6mbFcf',
 'Content-Type': 'application/json'}

In [ ]:
requests.patch(url, data=upload_json, headers=headers)

<Response [422]>

In [ ]:
upload_json='{"records": [{"fields": {"Proposition": "OEDEME", "id": "reczy8Os5Hc3qZ1yA"}}]}'

In [ ]:
requests.post(record_url, data=upload_json, headers=headers)

<Response [404]>

In [ ]:
def fmt_table(wb, nom_onglet):
  # onglet
  ws = wb.worksheet(nom_onglet)

  # Format de la partie gauche
  fmt = cellFormat(
      backgroundColor=color(0.91, 0.96, 0.93),
      textFormat=textFormat(bold=False, foregroundColor=color(0,0,0), fontSize='10'),
      horizontalAlignment='LEFT'
      )
  format_cell_range(ws, 'A:', fmt)

  # format de l'entete
  fmt = cellFormat(
      backgroundColor=color(0.7725,0.8431,0.7922),
      textFormat=textFormat(bold=True, foregroundColor=color(0,0,0), fontSize='10'),
      horizontalAlignment='LEFT'
      )
  format_cell_range(ws, '1', fmt)
  
  # format de l'auto wrap
  fmt = cellFormat(
      wrapStrategy='WRAP'
      )
  format_cell_range(ws, 'B:C', fmt)

  # on gèle l'entete et les colonnes de gauche
  set_frozen(ws, rows=1, cols=1)
  set_column_width(ws, 'A:C', 500)


In [ ]:
nom_onglet = 'Liste'
# si l'onglet existe déjà
try:
  ws = wb.worksheet(nom_onglet)
  wb.del_worksheet(ws)
except:
  print ("Onglet inexistant !")

wb.add_worksheet(nom_onglet, 1, 1)
export_sheet = wb.worksheet(nom_onglet)
set_with_dataframe(export_sheet, df)
fmt_table(wb,nom_onglet)


In [ ]:
table_name = 'Structure'
url = "https://api.airtable.com/v0/" + base_id + "/" + table_name

In [ ]:
df

,Name,Situation de départ,Notes
recnI80hvydv71fs4,0.Conception,"Plus on possède, plus on est soi-même possédé....","Trouver le plaisir d’écrire. Partir d’idées, d..."
recI2pgkNq2iP52rm,1.Situation initiale,Un jour d'été où il pleut et fait un temps abo...,Une vengeance d'une femme éconduite lors d'un ...
recb2DqhB9GvfI12n,2.Personnage,Un homme (marié ?) qui a donné rendez-vous à u...,Un Dom Juan sûr de son charme et imbu de sa pe...
recNLdU01JpCnvknJ,3.Acte déclencheur,La femme lui propose un jeu. Elle est dans le ...,NaN
recRxZVhpSPD6oQXx,4.Réaction,L'homme est d'abord tenté de partir car il est...,NaN
recRr06fOM4hGGaLY,5.Péripéties,"Pendant la visite (ennuyeuse), l'homme tente d...",NaN
recvWnAYn9VNySVlB,6.Chute,Un spectacle où une partie du public est appel...,NaN
recDmZtMMNmiy7R0M,Récit,NaN,NaN
